In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from random import randint
#  from scipy.stats import bernoulli

In [ ]:
class node1:
    def __init__(self, call, convert):
        self.stock_price = 50
        self.bond_value = -1
        self.coupon = 0 # the value of coupon at the current period
        # if coupon<0, apply a floor and let it be 0
        self.interest_rate = -1
        self.p_tilde = -1
        self.time_period = 0
        self.called = call # -1: not callable, 0: callable and not called, 1: callable and called 
        self.converted = convert
        self.bankrupt = 0 # 0: not bankrupt, 1: bankrupty now or before 
        self.up_parent = None
        self.down_parent = None
        self.up_child = None
        self.down_child = None
        self.path = []
        self.converted_val = -1 # the value of the bond after conversion

    # when considering convertible ones, please use this function to get the bond's value 
    def get_val(self):
        if(self.converted==1): return self.converted_val
        else: return self.bond_value

    # check if the path are all heads
    def all_head(self):
        if(self==None): return False
        for i in self.path: 
            if(i=="T"): return False
        return True
    
    def all_tail(self):
        if(self==None): return False
        for i in self.path: 
            if(i=="H"): return False
        return True
    
    # check that the info stored at the node is valid and updated
    def check_node(self):
        if(self==None): return True
        if(self.stock_price==0 and (self.brankrupt!=1)): return False
        if((self.stock_price!=0) and self.brankrupt==1): return False
        if(self.bankrupt==1 and (self.up_child!=None or self.down_child != None)): return False
        if(self.called==1 and (self.up_child != None or self.down_child != None)): return False

        if(self.up_parent!=None and self.down_parent!=None and (self.up_parent).converted==1 
           and (self.down_parent.converted==1) and (self.converted!=1 or self.coupon!=0)): return False
        if(self.up_parent==None and self.down_parent!=None
           and (self.down_parent.converted==1) and (self.converted!=1 or self.coupon!=0)): return False
        if(self.up_parent!=None and self.down_parent==None
           and (self.converted!=1 or self.coupon!=0)): return False
        return True


    # update this node and break from its children
    def bankrupt(self):
        self.stock_price = 0
        self.coupon = 0 
        self.p_tilde = -1
        self.bankrupt = 1 # 0: not bankrupt, 1: bankrupty now or before 
        self.up_child = None
        self.down_child = None
        self.update_node()

    # another version
    # def bankrupt(self, both):
    #     # both==1: update both childs
    #     # both==0: update only down_child
    #     # both==2: unknown
    #     self.stock_price = 0
    #     self.coupon = 0 
    #     self.p_tilde = -1
    #     self.bankrupt = 1 # 0: not bankrupt, 1: bankrupty now or before 
    #     if (both==1): 
    #         self.bankrupt(self.up_child, 1) # should I make them bankrupt or NaN?
    #         self.bankrupt(self.down_child)
    #     elif ((both==2) and (self.all_head())):
    #         self.bankrupt(self.up_child, 1)
    #         self.bankrupt(self.down_child)
    #     else: 
    #         self.bankrupt(self.up_child, 0)

        
    # update this node and break from its children
    def call(self, call_price):
        self.called = 1 # -1: not callable, 0: callable and not called, 1: callable and called 
        self.bong_value = call_price
        self.up_child = None
        self.donw_child = None
        self.check_node()

    # update this node
    def convert(self):
        self.converted = 1
        self.converted_val = self.stock_price
        self.check_node()

    # check this: when both paretns are converted, need to update 

    def print_node(self):
        print("print something about the node")

    def print_tree(self):
        print("print current node and following branches")

In [ ]:
def simulate_single(x):
    S0=30
    r=0.06
    t=12 # number of periods
    u=1+r+math.sqrt((1+r)**2-1) 
    drift=0
    vol=x # for additive amount, how much to add
    d=(1+r)/u
    face=50
    coupon_rate = 0.04
    coupon = coupon_rate*face  
    default_threshold = 1/3
    bankrupcy_ratio = 2.4
    K=55 # call value
    
    def g(x): # payoff function e.g for a call it would be max(0, x-K)
        if x==0:
            return [0, 0]
        if x <= default_threshold*S0:
            return [x*bankrupcy_ratio, 0]
        return [face, coupon] # first entry is the actual value, second is dividends or coupons
    
    # p=0.5#(1+r-d)/(u-d)
    # q=1-p
    # p=1-q # fixes floating-point errors hopefully
    # print(p, q) # only use these in the multiplicative model
    
    def nSigFigs(num, n):
        #print(num == np.nan)
        if type(num) not in [float, int, np.float64] or num == float("NaN"): return float("NaN")
        return int(num * 10**n) / 10**n
    
    def displayChart(tab):
        Chart=[]
        columns = []
        N = len(tab)-1
        for i in range(N, -1, -1):
            chart = []
            columns.append(N-i)
            for j in range(0,i):
                chart.append("")
            for j in range(i, N+1):
                entry = tab[j][i]
                chart.append(entry)
            Chart.append(chart)
        df = pd.DataFrame(Chart, columns = columns)
        return df.style.hide_index()
    
    def r_values(drift, vol):
        R = {}
        for i in range(t, -1, -1):
            R[i] = r + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0, i+1, 1)
        return R
    
    rates = r_values(0, 0.005)
    # print("interest rates")
    # displayChart(rates)

    interests = {}
    for key in range(0, t+1, 1):
        interests[key] = [0]*(key+1)
        for j in range(0, key+1, 1):
            index = randint(0, key)
            interests[key][j] = rates[key][index]

    
    def StockPrice(mode='a'): # a for additive, m for multiplicative
        StockPrice={}
        if mode == 'm':
            for i in range(t, -1, -1):
                StockPrice[i] = S0 * d ** (np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #np.empty(N+1)

        elif mode == 'a':
            for i in range(t, -1, -1):
                StockPrice[i] = np.maximum(np.zeros(i+1), S0 + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0,i+1,1))

        return StockPrice

    SP = StockPrice()
    # print("stock price")
    # displayChart(SP)
    
    def p_values(stock_price_chart, interests):
        P = {i: [1]*(i+1) for i in range(t+1)}
        for i in range(0, t):
            for j in range(i+1):
                if stock_price_chart[i][j] == 0:
                    P[i][j] = -1
                else:
                    d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                    u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                    P[i][j] = (1 + interests[i][j] - d) / (u-d)
        return P

    p_vals = p_values(SP, interests)
    # print("p tilde")
    # displayChart(p_vals)
    
    def exp(x,y,p,r):
        if p != -1:
            return (x*p+y*(1-p))/(1+r)
        return 0

    # multiplicative model
    def rnExp(x,y):
        return exp(x,y,p,r)
    
        
    bond_payoff = list(map(g, SP[t]))
    def vanillaBond(payoff): # coupon amount
        vanillaBond={}
        vanillaBond[t]=bond_payoff
        #IP=IntrinsicPut()
        #for n in range(0, N+1):
        #    PutPriceA[N].append(IP[N][n])
        for i in range(t-1, -1, -1):
            vanillaBond[i]=[]
            for j in range(i+1):
                entry = [0,0]
                if SP[i][j] != 0: 
                    entry[0] = exp(vanillaBond[i+1][j+1][0], vanillaBond[i+1][j][0], p_vals[i][j], interests[i][j]) + exp(vanillaBond[i+1][j+1][1], vanillaBond[i+1][j][1], p_vals[i][j], interests[i][j])
                    entry[0] = entry[0]
                    entry[1] = coupon if SP[i][j] >= default_threshold or i<t else 0
                #print(entry)
                vanillaBond[i].append(entry)
        return vanillaBond

    vBond = vanillaBond(bond_payoff)
    # print("Vanilla Bond payoff: [face value (after coupon), coupon]")
    # displayChart(vBond)

    def convBond(payoff):
        convBond = {}
        convBond[t] = [[max(SP[t][i], payoff[i][0]), payoff[i][1]] for i in range(t+1)]
        for i in range(t-1, -1, -1):
            convBond[i]=[]
            for j in range(i+1):
                entry = [0,0]
                if SP[i][j] != 0: 
                    entry[0] = exp(convBond[i+1][j+1][0], convBond[i+1][j][0], p_vals[i][j], interests[i][j]) \
                            + exp(convBond[i+1][j+1][1], convBond[i+1][j][1], p_vals[i][j], interests[i][j])
                    entry[0] = max(entry[0], SP[i][j])
                    entry[1] = coupon if SP[i][j] >= default_threshold or i<t else 0
                #print(entry)
                convBond[i].append(entry)
        return convBond

    cBond = convBond(bond_payoff)
    # print("Convertible Bond payoff: [face value (after coupon), coupon]")
    # print("Convertible but not callable")
    # displayChart(cBond)
    
    v_val = vBond[0][0]
    c_val = cBond[0][0]
    # print(v_val, c_val)
    return [v_val[0]+v_val[1], c_val[0]+c_val[1]]


simulate_single(5)

In [ ]:
runs = 5 # number of simulations at each vol (how much to increase/decrease in the additive model)
lo = 2 # lowest vol
hi = 8 # highest vol
incr = -0.1

keys = np.arange(hi, lo, incr)
v_price = {}
c_price = {}

for i in keys: # try different vol 
    v_price[i] = ([0]*runs)
    c_price[i] = ([0]*runs)
    for j in range(0, runs, 1): 
        (v_price[i][j], c_price[i][j]) = simulate_single(i)


In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot()

for key in v_price:
    if(key == lo or key == hi):
        ax1.scatter([key]*runs, v_price[key], s=10, c='b', marker="s", label="vanilla")
        ax1.scatter([key]*runs, c_price[key], s=10, c='r', marker="o", label="convertible")
    else:
        ax1.scatter([key]*runs, v_price[key], s=10, c='b', marker="s")
        ax1.scatter([key]*runs, c_price[key], s=10, c='r', marker="o")
    
plt.legend(loc='upper left')
plt.show()
                